In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

# Load .env into environment
load_dotenv()

# Pull the API key from .env 
TMDB_API_KEY = os.getenv("API_KEY")

film_ratings_df = pd.read_csv("../inputs/ratings.csv")

In [2]:
film_ratings_df

,Date,Name,Year,Letterboxd URI,Rating
0,2015-09-19,The Departed,2006,https://boxd.it/28MA,5.0
1,2015-09-19,John Wick,2014,https://boxd.it/7dEc,4.0
2,2015-09-19,Spartacus,1960,https://boxd.it/29bq,4.0
3,2015-09-19,Paths of Glory,1957,https://boxd.it/29b6,5.0
4,2015-09-19,The Sixth Sense,1999,https://boxd.it/29Js,4.5
...,...,...,...,...,...
415,2025-10-04,American Made,2017,https://boxd.it/bb66,3.5
416,2025-12-14,Wake Up Dead Man,2025,https://boxd.it/uAKS,3.5
417,2025-12-17,Kill Bill: Vol. 1,2003,https://boxd.it/70w,3.5
418,2025-12-19,Kill Bill: Vol. 2,2004,https://boxd.it/2au4,3.0
